In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov  4 11:49:40 2019

@author: ewell
"""
# Corey Schafer
# https://www.youtube.com/watch?v=ng2o98k983k

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
#url_base ="https://www.etsy.com/c/home-and-living?ref=pagination&page="

# Drawing and Illustrations
url_base ="https://www.etsy.com/c/art-and-collectibles/drawing-and-illustration?ref=pagination&explicit=1&page="

# Prints
#url_base ="https://www.etsy.com/c/art-and-collectibles/prints?explicit=1&ref=pagination&page="

# R: how many page listings to procesess; capping it for time
R = 24
urls=[]
for r in range(R): 
    urls.append(url_base+str(r+2))
    
#urls = ["https://www.etsy.com/c/home-and-living?ref=pagination&page=2",
#        "https://www.etsy.com/c/home-and-living?ref=pagination&page=3",
#        "https://www.etsy.com/c/home-and-living?ref=pagination&page=4",
#        "https://www.etsy.com/c/home-and-living?ref=pagination&page=5"]

page_count = 0

id = []
href = []
price = []
blurb = []
seller = []
stars = []
li_list = []
numrate = []
list_dict=[]

In [3]:
len(urls)

24

In [4]:
# Let's look at a page at random
url = urls[3]

# Request the page
request_result = requests.get(url)
#source = request_result.text

if request_result.status_code == 200:
    soup = bs(request_result.text, "lxml")

# Use Beautiful soup to get the HTML info (the "soup")
#soup = bs(source, "lxml")
#print(soup.prettify())  

element = soup.find("ul",{"class":"responsive-listing-grid wt-grid wt-grid--block justify-content-flex-start pl-xs-0"})    
li_match = element.find_all("li")
item = li_match[3]

image_path = str(item.find("img",{"class":"width-full wt-height-full display-block position-absolute"})["src"])
#img_i = str(image.find("div",{"class":"js-merch-stash-check-listing"})["data-palette-listing-id"])
            
#images = soup.find("img",{"class":"responsive-listing-grid wt-grid wt-grid--block justify-content-flex-start pl-xs-0"})    
#li_match = element.find_all("li")
print(image_path)
image_path_clean = image_path.split("?")[0]
print(image_path_clean)

https://i.etsystatic.com/11612604/c/2965/2354/0/373/il/0f2627/1673811890/il_340x270.1673811890_q1zq.jpg
https://i.etsystatic.com/11612604/c/2965/2354/0/373/il/0f2627/1673811890/il_340x270.1673811890_q1zq.jpg


In [5]:
import urllib.request as req
req.urlretrieve(image_path_clean, "00000002.jpg")

('00000002.jpg', <http.client.HTTPMessage at 0x2637c6f4668>)

In [6]:
# Loop through all Page Listing
for url in urls:
    page_count+=1
    
    # Request the page
    source = requests.get(url).text
    
    # Use Beautiful soup to get the HTML info (the "soup")
    soup = bs(source, "lxml")
    #print(soup.prettify())              
    
    #article = soup.find("ul",data-target="category-block-grid", class_="list-unstyled block-grid-xs-2 show-xs show-sm show-md  hide-lg hide-xl hide-tv")
    element = soup.find("ul",{"class":"responsive-listing-grid wt-grid wt-grid--block justify-content-flex-start pl-xs-0"})    
    li_match = element.find_all("li")
    
    #for li in range(150,490):
    li_count=0
    for item in li_match:#[0:20]:   
        li_count+=1
        try: 
            id_i = str(item.find("div",{"class":"js-merch-stash-check-listing"})["data-palette-listing-id"])
            #id.append(id_i)
            
            href_i = str(item.find("a",{"class":"display-inline-block listing-link"})["href"])
            #href.append(href_i)
            
            seller_i = str(item.find("div",{"class":"v2-listing-card__shop"}).p.text)
            #seller.append(seller_i)
            #seller.update({li:seller_i})
            
            # get current price
            price_i = str(item.find("span", {"class":"currency-value"}).text)
            #price.append(price_i)
            #price.update({li:price_i})
        
            # get blurb
            blurb_i = str(item.find("h2",{"class":"text-gray text-truncate mb-xs-0 text-body"}).text) 
            blurb_i = blurb_i.replace("\n","").strip()
            #blurb.update({li:blurb_i})
            
            stars_i = str(item.find("input",{"name":"rating"})["value"]) 
            #stars.append(stars_i)
    
            numrate_i = str(item.find("span",{"class":"text-body-smaller text-gray-lighter display-inline-block icon-b-1"}).text) 
            numrate_i=numrate_i.strip("()").replace(",","")
            #numrate.append(numrate_i)        
            #text-body-smaller text-gray-lighter display-inline-block icon-b-1
            
            image_path_i = str(item.find("img",{"class":"width-full wt-height-full display-block position-absolute"})["src"])
            image_path_clean_i = image_path_i.split("?")[0]
            
            row= {'id':id_i,'href':href_i,'seller':seller_i,'price':price_i,'blurb':blurb_i,
                  'stars':stars_i,'numrate':numrate_i,'im_path':image_path_i}
            list_dict.append(row)
            #row = (id_i,href_i,seller_i,price_i,blurb_i,stars_i,numrate_i)
            #print(row)
            #li_list.append(li)
        except Exception as e:
            break
            #id.append("none")
            #href.append("none")
            #seller.append("none")
            #price.append("none")
            #blurb.append("none")
            #stars.append("none")
            #numrate.append("none")
            #li_list.append(li)
    

In [7]:
#Create DataFrame from all the catagory lists
df = pd.DataFrame(list_dict)
df.set_index('id',inplace=True)
#df.to_csv("etsy_prints.csv")
#df = pd.DataFrame({"li_num":li_list})
# =============================================================================
# df = pd.DataFrame({"id":id})
#      
# #df["li_num"] = li_list
# df["href"] = href
# df["price"] = price
# df["blurb"] = blurb
# df["seller"] = seller
# df["stars"] = stars
# df["numrate"] = numrate
# =============================================================================

In [10]:
df.sample(4)

,blurb,href,im_path,numrate,price,seller,stars
id,,,,,,,
384757194,"Printable card, French quote card, Love card, ...",https://www.etsy.com/listing/384757194/printab...,https://i.etsystatic.com/12394010/c/1175/933/0...,137,1.44,MonRosePompon,5
527143006,"Green and Teal Glitter Digital Papers, Scrapbo...",https://www.etsy.com/listing/527143006/green-a...,https://i.etsystatic.com/15125539/c/700/555/0/...,360,0.99,ChubbyEyesDigital,4.9084
481478699,Dog Portrait Custom Sketch,https://www.etsy.com/listing/481478699/dog-por...,https://i.etsystatic.com/5463640/c/855/679/0/8...,60,55.00,SketchbookGallery,5
497950786,"Black and white quote poster, Modern design, H...",https://www.etsy.com/listing/497950786/black-a...,https://i.etsystatic.com/12394010/c/1175/933/0...,137,4.72,MonRosePompon,5


In [14]:
df.index.max()

'98573738'

In [15]:
df.index.min()

'101291064'